In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
# Utility for displaying our DataFrames
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [24]:
unclean = pd.read_csv('../match-data/uncleaned_intakes.csv')
clean = pd.DataFrame()

unclean.replace(['---'], '')

unclean['form.int.nation'] = unclean['form.int.nation'].replace(['1'], 'jordanian')
unclean['form.int.nation'] = unclean['form.int.nation'].replace(['2'], 'syrian')
unclean['form.int.nation'] = unclean['form.int.nation'].replace(['---'], '')

unclean['form.dem.male'] = unclean['form.dem.male'].replace(['0'], 'female')
unclean['form.dem.male'] = unclean['form.dem.male'].replace(['1'], 'male')

unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['1'], 'none')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['2'], 'primary')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['3'], 'secondary')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['4'], 'college')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['5'], 'diploma')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['6'], 'bachelors')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['7'], 'masters')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['7'], 'doctorate')
unclean['form.ed.edu1'] = unclean['form.ed.edu1'].replace(['---'], '')

unclean['form.int.nationality'] = unclean['form.int.nationality'].replace(['---'], '')
clean['nationalty'] = unclean['form.int.nationality'].map(str) + unclean['form.int.nation']

clean

,nationalty
0,syrian
1,syrian
2,syrian
3,syrian
4,syrian
5,syrian
6,syrian
7,syrian
8,syrian
9,syrian
